# UnbiasedGBM

:::{card} Unbiased gradient boosting decision tree with unbiased feature importance. (IJCAI2023)

- page: [Zhang, Z., Zhang, T., & Li, J. (2023, August). Unbiased gradient boosting decision tree with unbiased feature importance. In Proceedings of the Thirty-Second International Joint Conference on Artificial Intelligence (pp. 4629-4637).](https://dl.acm.org/doi/abs/10.24963/ijcai.2023/515)
- pdf: [arxiv.org/pdf/2305.10696v1](https://arxiv.org/pdf/2305.10696v1)

:::

## 概要

GBDTの分割（split）を決めるアルゴリズムには次のような バイアス（偏り） が存在することが問題になっている：

1. 特徴量の 分割候補の数（例えば連続値やカテゴリ数が多い特徴は多くの分割点を持てる）に依存して、有利になりやすい。
2. 分割の「改善度」（gain）を推定する際に、訓練データだけを使って評価するため、過学習や誤った重要度評価につながる。

この偏りにより、以下のような問題が起きる：

1. 解釈性（interpretability）が低くなる：実際には無関係な特徴量が高く評価されてしまう可能性。 
2. 過学習（overfitting）の原因となる：訓練データ上でよくなる分割を選びがちで、それが一般化性能を損なうことがある。 

これらの問題に対処するために、Unbiased GBMを提唱